In [1]:
import numpy as np
import pandas as pd

In [6]:
import plotly.graph_objects as go

# HODP colors
monochrome_colors = ['#251616', '#760000', '#C63F3F', '#E28073', '#F1D3CF']
primary_colors = ['#C63F3F', '#F4B436', '#83BFCC', '#455574', '#E2DDDB']

# HODP template
theme_hodp = go.layout.Template(
    layout=go.Layout(
        title = {'font':{'size':24, 'family':"Helvetica", 'color':monochrome_colors[0]}, 'pad':{'t':100, 'r':0, 'b':0, 'l':0}},
        font = {'size':18, 'family':'Helvetica', 'color':'#717171'},
        xaxis = {'ticks': "outside",
                'tickfont': {'size': 14, 'family':"Helvetica"},
                'showticksuffix': 'all',
                'showtickprefix': 'last',
                'showline': True,
                'title':{'font':{'size':18, 'family':'Helvetica'}, 'standoff':20},
                'automargin': True
                },
        yaxis = {'ticks': "outside",
                'tickfont': {'size': 14, 'family':"Helvetica"},
                'showticksuffix': 'all',
                'showtickprefix': 'last',
                'title':{'font':{'size':18, 'family':'Helvetica'}, 'standoff':20},
                'showline': True,
                'automargin': True
                },
        legend = {'bgcolor':'rgba(0,0,0,0)', 
                'title':{'font':{'size':18, 'family':"Helvetica", 'color':monochrome_colors[0]}}, 
                'font':{'size':14, 'family':"Helvetica"}, 
                'yanchor':'bottom'
                },
        colorscale = {'diverging':monochrome_colors},
        coloraxis = {'autocolorscale':True, 
                'cauto':True, 
                'colorbar':{'tickfont':{'size':14,'family':'Helvetica'}, 'title':{'font':{'size':18, 'family':'Helvetica'}}},
                }
    )
)

In [8]:
import numpy as np
import pandas as pd

vdistances = pd.read_csv("geocoded_businesses.csv")
vdistances.head(5)

,Unnamed: 0,Name,Latitude,Longitude,Address,Storefront Sales Status,Sales Options,Grocery Sales,Phone,Website,Facebook,Twitter,Instagram,LastModified
0,0,1369 Coffee House,42.390604,-71.121712,NaN,Open,"Delivery, Take Out",yes,757 Massachusetts Ave,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse?ref_src=tw...,https://www.instagram.com/1369coffeehouse/?hl=en,2020-06-16T17:08:30Z
1,1,1369 Coffee House,42.373793,-71.100367,NaN,Open,"Delivery, Take Out, Outdoor Dining",NaN,1369 Cambridge Street,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse,https://www.instagram.com/1369coffeehouse/?hl=en,2020-07-10T15:24:36Z
2,2,2 4-1 Optical,42.390604,-71.121712,NaN,Open,NaN,NaN,1128 Massachusetts Ave,NaN,https://www.facebook.com/241optical/,NaN,NaN,2020-06-16T17:09:04Z
3,3,5th Street Laundromat,42.369638,-71.083565,NaN,Open,NaN,NaN,58 5th Street,NaN,NaN,NaN,NaN,2020-03-22T12:45:32Z
4,5,730 Tavern,42.366016,-71.105142,NaN,Open,"Delivery, Take Out, Indoor Dining",NaN,730 Massachusetts Ave,NaN,https://www.facebook.com/730tavern/,NaN,https://www.instagram.com/p/B9uqJpAgWJz/,2020-12-15T13:55:02Z


In [9]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [10]:
harv_lat = 42.3744
harv_long = -71.1171
mit_lat = 42.3598
mit_long = -71.0921

vdistances["harv_dist"] = ""
vdistances["mit_dist"] = ""
vdistances["min_dist"] = ""

for i in range(len(vdistances.index)):
    vdistances["harv_dist"][i] = haversine(harv_long, harv_lat, vdistances["Longitude"][i], vdistances["Latitude"][i])
    vdistances["mit_dist"][i] = haversine(mit_long, mit_lat, vdistances["Longitude"][i], vdistances["Latitude"][i])
    vdistances["min_dist"][i] = min(vdistances["harv_dist"][i], vdistances["mit_dist"][i])

vdistances.head()
vdistances.to_csv("vdistances.csv")

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [33]:
openloc = vdistances[(vdistances["Storefront Sales Status"] == "Open")]
closedloc = vdistances[(vdistances["Storefront Sales Status"] == "Closed")]
closedloc.head(5)



,Unnamed: 0,Name,Latitude,Longitude,Address,Storefront Sales Status,Sales Options,Grocery Sales,Phone,Website,Facebook,Twitter,Instagram,LastModified,harv_dist,mit_dist,min_dist
9,10,Abide Place,42.390604,-71.121712,NaN,Closed,NaN,NaN,317 Massachusetts Ave,NaN,NaN,NaN,https://www.instagram.com/abidecambridge/,2020-11-23T14:11:22Z,1.841192,4.201103,1.841192
31,33,American Dural,43.366228,-80.322486,NaN,Closed,NaN,NaN,135 Huron St.,NaN,NaN,NaN,NaN,2020-09-23T21:49:05Z,757.846297,760.183302,757.846297
43,46,Apple Cinemas,42.392738,-71.140817,NaN,Closed,NaN,NaN,168 Alewife Brook Pkwy,NaN,NaN,NaN,NaN,2020-06-09T17:31:30Z,2.820016,5.424803,2.820016
50,53,Art and Soul Yoga,42.372840,-71.099755,Online Sales,Closed,NaN,NaN,91 Hampshire St.,NaN,NaN,NaN,NaN,2020-06-09T17:31:30Z,1.43536,1.580498,1.43536
72,77,Beat Brew Hall,42.373357,-71.119896,NaN,Closed,NaN,NaN,13A Brattle St,NaN,https://www.facebook.com/beatbrewhall/,NaN,https://www.instagram.com/beatbrewhall/,2020-06-09T17:49:22Z,0.257302,2.736304,0.257302


In [12]:
import matplotlib.pyplot as plt
plt.scatter(openloc["mit_dist"],openloc["harv_dist"],c='#2596be')
plt.xlabel("Distance to MIT")
plt.ylabel("Distance to Harvard")
plt.title("Distance to Campus and 1-2 Year Vacancies")
plt.show()

In [34]:
openmin = []
openmin = np.append (openmin, (vdistances.loc[vdistances["Storefront Sales Status"]== "Open"]["min_dist"]))
closedmin=[]
closedmin=np.append (openmin, (vdistances.loc[vdistances["Storefront Sales Status"]== "Closed"]["min_dist"]))
openY=[]
closedY=[]

openY = np.ones((len(openmin),), dtype=int)
closedY = np.zeros((len(closedmin),),dtype=int)

closedminfinal=[]
for i in range(len(closedmin)):
    if closedmin[i]<5:
        closedminfinal.append(closedmin[i])
openminfinal=[]
for i in range(len(openmin)):
    if openmin[i]<5:
        openminfinal.append(openmin[i])


In [16]:
# HODP colors
monochrome_colors = ['#251616', '#760000', '#C63F3F', '#E28073', '#F1D3CF']
primary_colors = ['#C63F3F', '#F4B436', '#83BFCC', '#455574', '#E2DDDB']

# HODP template
theme_hodp = go.layout.Template(
    layout=go.Layout(
        title = {'font':{'size':24, 'family':"Helvetica", 'color':monochrome_colors[0]}, 'pad':{'t':100, 'r':0, 'b':0, 'l':0}},
        font = {'size':18, 'family':'Helvetica', 'color':'#717171'},
        xaxis = {'ticks': "outside",
                'tickfont': {'size': 14, 'family':"Helvetica"},
                'showticksuffix': 'all',
                'showtickprefix': 'last',
                'showline': True,
                'title':{'font':{'size':18, 'family':'Helvetica'}, 'standoff':20},
                'automargin': True
                },
        yaxis = {'ticks': "outside",
                'tickfont': {'size': 14, 'family':"Helvetica"},
                'showticksuffix': 'all',
                'showtickprefix': 'last',
                'title':{'font':{'size':18, 'family':'Helvetica'}, 'standoff':20},
                'showline': True,
                'automargin': True
                },
        legend = {'bgcolor':'rgba(0,0,0,0)', 
                'title':{'font':{'size':18, 'family':"Helvetica", 'color':monochrome_colors[0]}}, 
                'font':{'size':14, 'family':"Helvetica"}, 
                'yanchor':'bottom'
                },
        colorscale = {'diverging':monochrome_colors},
        coloraxis = {'autocolorscale':True, 
                'cauto':True, 
                'colorbar':{'tickfont':{'size':14,'family':'Helvetica'}, 'title':{'font':{'size':18, 'family':'Helvetica'}}},
                }
    )
)

In [31]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Histogram(x=closedminfinal, marker_color=primary_colors[0], name='Closed'))
fig.add_trace(go.Histogram(x=openminfinal, marker_color=primary_colors[1], name='Open'))

#Overlaid Histogram
fig.update_layout(title = "Comparison of Open and Closed by Minimum Distance to School", barmode='overlay')
fig.update_traces(opacity=0.7) # Reduce opacity to see both histograms
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf414bac-1baa-463f-8d73-f1d9a87e25d5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>